---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [582]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind as ttest

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [382]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [622]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    import re
    with open('university_towns.txt') as f:
        lines = f.readlines()
    for i in range(len(lines)):
        if 'edit' not in lines[i]:
            lines[i]=re.sub('\n',"",re.sub(r' \(.*',"",lines[i]))
    state_index=[]
    for i in range(len(lines)):
        if 'edit' in lines[i]:
            state_index.append(i)
    d=dict()
    for i in range(len(lines)):
        for j in range(len(state_index)-1):
            if (i>=state_index[j]) & (i<state_index[j+1]):
                d[re.sub('\n',"",re.sub(r'\[.*',"",lines[i]))]=re.sub('\n',"",re.sub(r'\[.*',"",lines[state_index[j]]))
    #df1=pd.DataFrame.from_dict(d.items())
    #df1=pd.DataFrame(eval(d))
    df1=pd.DataFrame.from_dict(d,orient='index')
    #df1['RegionName']=df1.index
    df1.reset_index(inplace=True)
    df1.columns=['RegionName','State']
    #df1.rename.columns=['RegionName','State']
    
    
    df1.sort_values('State',ascending= True, inplace=True)
    df1.reset_index(inplace=True)
    df1=df1[['State','RegionName']]
    df2=df1.where(df1['State'])

    return df1

In [260]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    import numpy as np
    
    df_gdp=pd.read_excel('gdplev.xls',skiprows=5,Header=0)
    df_gdp=df_gdp.loc[df_gdp['Unnamed: 4'] >= '2000']
    df_gdp=df_gdp[['Unnamed: 4','GDP in billions of current dollars.1','GDP in billions of chained 2009 dollars.1']]
    df_gdp.rename(columns={'Unnamed: 4': 'Quarters', 'GDP in billions of current dollars.1': 'GDP in billions','GDP in billions of chained 2009 dollars.1':'Chained GDP in billions'}, inplace=True)
    df_gdp['sign']=np.sign(df_gdp['Chained GDP in billions'].diff().fillna(0))
    start=[]
    for i in range(len(df_gdp['sign'])-1):
        if (df_gdp['sign'].iloc[i] == -1.0) & (df_gdp['sign'].iloc[i+1] == -1.0):
            start.append(df_gdp['Quarters'].iloc[i])
    recession_start=start[0]

    return str(recession_start)


In [290]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    import numpy as np
    
    df_gdp=pd.read_excel('gdplev.xls',skiprows=5,Header=0)
    df_gdp=df_gdp.loc[df_gdp['Unnamed: 4'] >= '2000']
    df_gdp=df_gdp[['Unnamed: 4','GDP in billions of current dollars.1','GDP in billions of chained 2009 dollars.1']]
    df_gdp.rename(columns={'Unnamed: 4': 'Quarters', 'GDP in billions of current dollars.1': 'GDP in billions','GDP in billions of chained 2009 dollars.1':'Chained GDP in billions'}, inplace=True)
    df_gdp['sign']=np.sign(df_gdp['Chained GDP in billions'].diff().fillna(0))
    start=[]
    for i in range(len(df_gdp['sign'])-1):
        if (df_gdp['sign'].iloc[i] == -1.0) & (df_gdp['sign'].iloc[i+1] == -1.0):
            start.append(df_gdp['Quarters'].iloc[i])
    recession_start=start[0]
    
    end=[]
    period1=df_gdp.where(df_gdp['Quarters'] > recession_start).dropna()
    for i in range(len(period1['sign'])-1):
        if (period1['sign'].iloc[i] == 1.0) & (period1['sign'].iloc[i+1] == 1.0):
            end.append(period1['Quarters'].iloc[i])
    recession_end=end[1]
       
    return str(recession_end)


In [315]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    import numpy as np
    
    df_gdp=pd.read_excel('gdplev.xls',skiprows=5,Header=0)
    df_gdp=df_gdp.loc[df_gdp['Unnamed: 4'] >= '2000']
    df_gdp=df_gdp[['Unnamed: 4','GDP in billions of current dollars.1','GDP in billions of chained 2009 dollars.1']]
    df_gdp.rename(columns={'Unnamed: 4': 'Quarters', 'GDP in billions of current dollars.1': 'GDP in billions','GDP in billions of chained 2009 dollars.1':'Chained GDP in billions'}, inplace=True)
    df_gdp['sign']=np.sign(df_gdp['Chained GDP in billions'].diff().fillna(0))
    start=[]
    for i in range(len(df_gdp['sign'])-1):
        if (df_gdp['sign'].iloc[i] == -1.0) & (df_gdp['sign'].iloc[i+1] == -1.0):
            start.append(df_gdp['Quarters'].iloc[i])
    recession_start=start[0]
    
    end=[]
    period1=df_gdp.where(df_gdp['Quarters'] > recession_start).dropna()
    for i in range(len(period1['sign'])-1):
        if (period1['sign'].iloc[i] == 1.0) & (period1['sign'].iloc[i+1] == 1.0):
            end.append(period1['Quarters'].iloc[i])
    recession_end=end[0]
    recession_period=df_gdp.where((df_gdp['Quarters'] >= recession_start) & (df_gdp['Quarters'] <= recession_end)).dropna()
    min_gdp=recession_period['Chained GDP in billions'].min()
    bottom=recession_period.loc[recession_period['Chained GDP in billions']==min_gdp,'Quarters'].values[0]
    
    return str(bottom)

In [627]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    acronym=pd.DataFrame.from_dict(states,orient='index')
    acronym.reset_index(inplace=True)
    acronym.rename(columns={'index': 'Stateabr', 0: 'State'}, inplace=True)
    housing_df=pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_df=pd.merge(housing_df,acronym, how='inner',  left_on=['State'],right_on=['Stateabr'])
    housing_df.rename(columns={'State_y':'State'},inplace=True)
    housing_df.set_index(['State', 'RegionName'], inplace=True)
    housing_df['2000q1']=housing_df[['2000-01','2000-02','2000-03']].mean(axis=1)
    housing_df['2000q2']=housing_df[['2000-04','2000-05','2000-06']].mean(axis=1)
    housing_df['2000q3']=housing_df[['2000-07','2000-08','2000-09']].mean(axis=1)
    housing_df['2000q4']=housing_df[['2000-10','2000-11','2000-12']].mean(axis=1)
    housing_df['2001q1']=housing_df[['2001-01','2001-02','2001-03']].mean(axis=1)
    housing_df['2001q2']=housing_df[['2001-04','2001-05','2001-06']].mean(axis=1)
    housing_df['2001q3']=housing_df[['2001-07','2001-08','2001-09']].mean(axis=1)
    housing_df['2001q4']=housing_df[['2001-10','2001-11','2001-12']].mean(axis=1)
    housing_df['2002q1']=housing_df[['2002-01','2002-02','2002-03']].mean(axis=1)
    housing_df['2002q2']=housing_df[['2002-04','2002-05','2002-06']].mean(axis=1)
    housing_df['2002q3']=housing_df[['2002-07','2002-08','2002-09']].mean(axis=1)
    housing_df['2002q4']=housing_df[['2002-10','2002-11','2002-12']].mean(axis=1)
    housing_df['2003q1']=housing_df[['2003-01','2003-02','2003-03']].mean(axis=1)
    housing_df['2003q2']=housing_df[['2003-04','2003-05','2003-06']].mean(axis=1)
    housing_df['2003q3']=housing_df[['2003-07','2003-08','2003-09']].mean(axis=1)
    housing_df['2003q4']=housing_df[['2003-10','2003-11','2003-12']].mean(axis=1)
    housing_df['2004q1']=housing_df[['2004-01','2004-02','2004-03']].mean(axis=1)
    housing_df['2004q2']=housing_df[['2004-04','2004-05','2004-06']].mean(axis=1)
    housing_df['2004q3']=housing_df[['2004-07','2004-08','2004-09']].mean(axis=1)
    housing_df['2004q4']=housing_df[['2004-10','2004-11','2004-12']].mean(axis=1)
    housing_df['2005q1']=housing_df[['2005-01','2005-02','2005-03']].mean(axis=1)
    housing_df['2005q2']=housing_df[['2005-04','2005-05','2005-06']].mean(axis=1)
    housing_df['2005q3']=housing_df[['2005-07','2005-08','2005-09']].mean(axis=1)
    housing_df['2005q4']=housing_df[['2005-10','2005-11','2005-12']].mean(axis=1)
    housing_df['2006q1']=housing_df[['2006-01','2006-02','2006-03']].mean(axis=1)
    housing_df['2006q2']=housing_df[['2006-04','2006-05','2006-06']].mean(axis=1)
    housing_df['2006q3']=housing_df[['2006-07','2006-08','2006-09']].mean(axis=1)
    housing_df['2006q4']=housing_df[['2006-10','2006-11','2006-12']].mean(axis=1)
    housing_df['2007q1']=housing_df[['2007-01','2007-02','2007-03']].mean(axis=1)
    housing_df['2007q2']=housing_df[['2007-04','2007-05','2007-06']].mean(axis=1)
    housing_df['2007q3']=housing_df[['2007-07','2007-08','2007-09']].mean(axis=1)
    housing_df['2007q4']=housing_df[['2007-10','2007-11','2007-12']].mean(axis=1)
    housing_df['2008q1']=housing_df[['2008-01','2008-02','2008-03']].mean(axis=1)
    housing_df['2008q2']=housing_df[['2008-04','2008-05','2008-06']].mean(axis=1)
    housing_df['2008q3']=housing_df[['2008-07','2008-08','2008-09']].mean(axis=1)
    housing_df['2008q4']=housing_df[['2008-10','2008-11','2008-12']].mean(axis=1)
    housing_df['2009q1']=housing_df[['2009-01','2009-02','2009-03']].mean(axis=1)
    housing_df['2009q2']=housing_df[['2009-04','2009-05','2009-06']].mean(axis=1)
    housing_df['2009q3']=housing_df[['2009-07','2009-08','2009-09']].mean(axis=1)
    housing_df['2009q4']=housing_df[['2009-10','2009-11','2009-12']].mean(axis=1)
    housing_df['2010q1']=housing_df[['2010-01','2010-02','2010-03']].mean(axis=1)
    housing_df['2010q2']=housing_df[['2010-04','2010-05','2010-06']].mean(axis=1)
    housing_df['2010q3']=housing_df[['2010-07','2010-08','2010-09']].mean(axis=1)
    housing_df['2010q4']=housing_df[['2010-10','2010-11','2010-12']].mean(axis=1)
    housing_df['2011q1']=housing_df[['2011-01','2011-02','2011-03']].mean(axis=1)
    housing_df['2011q2']=housing_df[['2011-04','2011-05','2011-06']].mean(axis=1)
    housing_df['2011q3']=housing_df[['2011-07','2011-08','2011-09']].mean(axis=1)
    housing_df['2011q4']=housing_df[['2011-10','2011-11','2011-12']].mean(axis=1)
    housing_df['2012q1']=housing_df[['2012-01','2012-02','2012-03']].mean(axis=1)
    housing_df['2012q2']=housing_df[['2012-04','2012-05','2012-06']].mean(axis=1)
    housing_df['2012q3']=housing_df[['2012-07','2012-08','2012-09']].mean(axis=1)
    housing_df['2012q4']=housing_df[['2012-10','2012-11','2012-12']].mean(axis=1)
    housing_df['2013q1']=housing_df[['2013-01','2013-02','2013-03']].mean(axis=1)
    housing_df['2013q2']=housing_df[['2013-04','2013-05','2013-06']].mean(axis=1)
    housing_df['2013q3']=housing_df[['2013-07','2013-08','2013-09']].mean(axis=1)
    housing_df['2013q4']=housing_df[['2013-10','2013-11','2013-12']].mean(axis=1)
    housing_df['2014q1']=housing_df[['2014-01','2014-02','2014-03']].mean(axis=1)
    housing_df['2014q2']=housing_df[['2014-04','2014-05','2014-06']].mean(axis=1)
    housing_df['2014q3']=housing_df[['2014-07','2014-08','2014-09']].mean(axis=1)
    housing_df['2014q4']=housing_df[['2014-10','2014-11','2014-12']].mean(axis=1)
    housing_df['2015q1']=housing_df[['2015-01','2015-02','2015-03']].mean(axis=1)
    housing_df['2015q2']=housing_df[['2015-04','2015-05','2015-06']].mean(axis=1)
    housing_df['2015q3']=housing_df[['2015-07','2015-08','2015-09']].mean(axis=1)
    housing_df['2015q4']=housing_df[['2015-10','2015-11','2015-12']].mean(axis=1)
    housing_df['2016q1']=housing_df[['2016-01','2016-02','2016-03']].mean(axis=1)
    housing_df['2016q2']=housing_df[['2016-04','2016-05','2016-06']].mean(axis=1)
    housing_df['2016q3']=housing_df[['2016-07','2016-08']].mean(axis=1)
    
    
    housing_df=housing_df[['2000q1','2000q2','2000q3','2000q4','2001q1','2001q2','2001q3','2001q4','2002q1','2002q2','2002q3','2002q4','2003q1','2003q2','2003q3','2003q4','2004q1','2004q2','2004q3','2004q4','2005q1','2005q2','2005q3','2005q4','2006q1','2006q2','2006q3','2006q4','2007q1','2007q2','2007q3','2007q4','2008q1','2008q2','2008q3','2008q4','2009q1','2009q2','2009q3','2009q4','2010q1','2010q2','2010q3','2010q4','2011q1','2011q2','2011q3','2011q4','2012q1','2012q2','2012q3','2012q4','2013q1','2013q2','2013q3','2013q4','2014q1','2014q2','2014q3','2014q4','2015q1','2015q2','2015q3','2015q4','2016q1','2016q2','2016q3']]
    return housing_df

In [ ]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"

In [541]:
recession_housing_prices=housing_df[['2008q3','2008q4','2009q1','2009q2']]
recession_housing_prices=recession_housing_prices.reset_index()

In [557]:
recession_housing_prices.head()

,State,RegionName,2008q3,2008q4,2009q1,2009q2
0,NY,New York,499766.666667,487933.333333,477733.333333,465833.333333
1,CA,Los Angeles,469500.000000,443966.666667,426266.666667,413900.000000
2,IL,Chicago,232000.000000,227033.333333,223766.666667,219700.000000
3,PA,Philadelphia,116933.333333,115866.666667,116200.000000,116166.666667
4,AZ,Phoenix,193766.666667,183333.333333,177566.666667,168233.333333


In [555]:
university_towns.head()

,State,RegionName
0,AL,Livingston
1,AL,Auburn
2,AL,Tuscaloosa
3,AL,Alabama
4,AL,Jacksonville


In [573]:
new_df=pd.merge(recession_housing_prices,university_towns, how='left', on='RegionName')

In [574]:
new_df

,State_x,RegionName,2008q3,2008q4,2009q1,2009q2,State_y
0,NY,New York,4.997667e+05,4.879333e+05,4.777333e+05,4.658333e+05,NY
1,CA,Los Angeles,4.695000e+05,4.439667e+05,4.262667e+05,4.139000e+05,NaN
2,IL,Chicago,2.320000e+05,2.270333e+05,2.237667e+05,2.197000e+05,NaN
3,PA,Philadelphia,1.169333e+05,1.158667e+05,1.162000e+05,1.161667e+05,NaN
4,AZ,Phoenix,1.937667e+05,1.833333e+05,1.775667e+05,1.682333e+05,NaN
5,NV,Las Vegas,2.133667e+05,1.949333e+05,1.812000e+05,1.643333e+05,NM
6,CA,San Diego,4.246667e+05,4.076333e+05,3.957000e+05,3.895000e+05,CA
7,TX,Dallas,1.121667e+05,1.099000e+05,1.076667e+05,1.051000e+05,TX
8,CA,San Jose,5.831333e+05,5.601000e+05,5.481667e+05,5.303000e+05,NaN
9,FL,Jacksonville,1.547333e+05,1.491667e+05,1.452667e+05,1.408333e+05,AL


In [586]:

new_df=new_df[['State_x','RegionName','2008q3','2008q4','2009q1','2009q2']]

In [567]:
non_univ_towns=new_df[np.isnan(new_df['2008q3'])&np.isnan(new_df['2008q4'])&np.isnan(new_df['2009q1'])&np.isnan(new_df['2009q2'])]

In [570]:
non_univ_towns=non_univ_towns[['State_x','RegionName']]

In [590]:
new_df1=pd.merge(recession_housing_prices,non_univ_towns, how='inner', on='RegionName').dropna()

In [591]:
new_df1.head()

,State,RegionName,2008q3,2008q4,2009q1,2009q2,State_x
0,OH,Columbus,112200.000000,111600.000000,110966.666667,109766.666667,GA
1,OH,Columbus,112200.000000,111600.000000,110966.666667,109766.666667,TX
4,IN,Columbus,125266.666667,124933.333333,123700.000000,121866.666667,GA
5,IN,Columbus,125266.666667,124933.333333,123700.000000,121866.666667,TX
6,MS,Columbus,120900.000000,123566.666667,127433.333333,129933.333333,GA


In [621]:
ttest(new_df['2008q3'](axis=0),new_df1['2008q3'](axis=0))

TypeError: 'Series' object is not callable